In [1]:
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
import preprocessingWithMissingvalues

carDf= preprocessingWithMissingvalues.preprocess('../ann/cardata.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15420 entries, 0 to 15419
Data columns (total 33 columns):
Month                   15420 non-null object
WeekOfMonth             15420 non-null int64
DayOfWeek               15420 non-null object
Make                    15420 non-null object
AccidentArea            15420 non-null object
DayOfWeekClaimed        15420 non-null object
MonthClaimed            15420 non-null object
WeekOfMonthClaimed      15420 non-null int64
Sex                     15420 non-null object
MaritalStatus           15420 non-null object
Age                     15420 non-null int64
Fault                   15420 non-null object
PolicyType              15420 non-null object
VehicleCategory         15420 non-null object
VehiclePrice            15420 non-null object
FraudFound              15420 non-null object
PolicyNumber            15420 non-null int64
RepNumber               15420 non-null int64
Deductible              15420 non-null int64
DriverRating           

/usr/local/anaconda35/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [3]:
#get the lable from the dataset
carDf.head()
carLable= carDf['Lable']
carDf.drop(['Lable'],inplace=True,axis=1) #drop the lable;
carDf[0:2]

,Make_Accura,Make_BMW,Make_Chevrolet,Make_Dodge,Make_Ferrari,Make_Ford,Make_Honda,Make_Jaguar,Make_Lexus,Make_Mazda,...,AddressChange-Claim_under 6 months,NumberOfCars_1 vehicle,NumberOfCars_2 vehicles,NumberOfCars_3 to 4,NumberOfCars_5 to 8,NumberOfCars_more than 8,RepNo,Deductible,DriverRating,DaysDiff
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0.733333,0.00,0.0,0.071795
1,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0.933333,0.25,1.0,0.069231


In [4]:
carDateNormalized= carDf

In [5]:
#Divide the data into fraudulent and non-fraudulent data
columnlable= list(carDateNormalized.columns.values)

#create the dataframe for fraudulent and non-fraudulent data
nonFraudulent= pd.DataFrame(columns=columnlable)
nonFraudulentLable= pd.DataFrame(columns=['lable'])

fraudulent= pd.DataFrame(columns=columnlable)
fraudulentLable= pd.DataFrame(columns=['lable'])


j= 0
k= 0
for i in range(carDateNormalized.shape[0]):
    if(carLable[i]==0):
        nonFraudulent.loc[j]= carDateNormalized.loc[i]
        nonFraudulentLable.loc[j]= 0.0
        j += 1
    else:
        fraudulent.loc[k]= carDateNormalized.loc[i]
        fraudulentLable.loc[i]= 1.0
        k += 1

In [6]:
print(nonFraudulent.shape)
print(fraudulent.shape)
nonFraudulent.head()

(14496, 97)
(923, 97)


,Make_Accura,Make_BMW,Make_Chevrolet,Make_Dodge,Make_Ferrari,Make_Ford,Make_Honda,Make_Jaguar,Make_Lexus,Make_Mazda,...,AddressChange-Claim_under 6 months,NumberOfCars_1 vehicle,NumberOfCars_2 vehicles,NumberOfCars_3 to 4,NumberOfCars_5 to 8,NumberOfCars_more than 8,RepNo,Deductible,DriverRating,DaysDiff
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.733333,0.00,0.000000,0.071795
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.933333,0.25,1.000000,0.069231
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.400000,0.25,0.666667,0.089744
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.200000,0.25,0.333333,0.107692
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.133333,0.25,0.000000,0.094872


In [7]:
#convert to dataframe
nonFraudulentLable= pd.DataFrame(nonFraudulentLable)
fraudulentLable= pd.DataFrame(fraudulentLable)


In [8]:
nonFraudulent.head()
fraudulentLable.head()

,lable
28,1.0
52,1.0
53,1.0
94,1.0
96,1.0


In [9]:
#divide the nonFraudulent into test and train
nonFraudX_train,nonFraudX_test,nonFraudY_train,nonFraudY_test = train_test_split(nonFraudulent,nonFraudulentLable,random_state=3,test_size=0.25)
print('nonFraudxtrain:',type(nonFraudX_train))
print(nonFraudX_test.shape)

#divide the trainset into trainset and validation set
nonFraudX_trainNew,nonFraudX_Valid,nonFraudY_trainNew,nonFraudY_Valid= train_test_split(nonFraudX_train,nonFraudY_train,random_state=3,test_size=0.2)

#divide the fraudulent into test and train
fraudX_train,fraudX_test,fraudY_train,fraudY_test = train_test_split(fraudulent, fraudulentLable,random_state=3,test_size=0.25)
print('fraudxtrain:',type(fraudX_train))
print(fraudX_test.shape)

nonFraudxtrain: <class 'pandas.core.frame.DataFrame'>
(3624, 97)
fraudxtrain: <class 'pandas.core.frame.DataFrame'>
(231, 97)


In [10]:
print(nonFraudX_trainNew.shape)
print(nonFraudX_Valid.shape)

(8697, 97)
(2175, 97)


In [11]:
#convert into numpy
nonFraudX_trainNew= nonFraudX_trainNew.values
nonFraudY_trainNew= nonFraudY_trainNew.values

nonFraudX_test= nonFraudX_test.values
nonFraudY_test= nonFraudY_test.values

fraudX_train= fraudX_train.values
fraudY_train= fraudY_train.values

fraudX_test= fraudX_test.values
fraudY_test= fraudY_test.values

nonFraudX_Valid= nonFraudX_Valid.values
nonFraudY_Valid= nonFraudY_Valid.values

In [12]:
#create the class Dataset which returns the data and labels
from torch.utils.data.dataset import Dataset, TensorDataset
class myDataset(Dataset):
    
    def __init__(self,features,lables,transform= None):
        dataTensor= []
        lableTensor= []
        dataSize= features.shape[0]
        
        for data in range(dataSize):
            feature= features[data,:]
            #feature= torch.from_numpy(feature).float()
            feature= torch.Tensor(feature)
            dataTensor.append(feature)
            
            lable= np.asanyarray(lables[data])
            lable= torch.from_numpy(lable).float()
           
            #lable= torch.Tensor(lable)
            #print("lable:",lable)
            #assert(False)
           
            lableTensor.append(lable)
        
        #put everything in features and lables
        self.features= dataTensor
        self.lables= lableTensor
    
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self,index):
        
        feature= self.features[index]
        lable= self.lables[index]
        #print("get_item feature:",feature)
        #print("get_item lable:",lable)
        return feature,lable

In [13]:
#make the  nonfraudulent dataset for train and test loader
myNonFraudulentTrainDataset= myDataset(nonFraudX_trainNew,nonFraudY_trainNew)
myNonFraudulentTestDataset= myDataset(nonFraudX_test,nonFraudY_test)

#make validation loader
myNonFraudulentValidation= myDataset(nonFraudX_Valid,nonFraudY_Valid)

#make the fraudulent dataset for train and test loader
myFraudulentTrainDataset= myDataset(fraudX_train,fraudY_train)
myFraudulentTestDataset= myDataset(fraudX_test,fraudY_test)

In [14]:
#make the trainloader and test loader for nonfraudulent dataset.
nonFraudulentTrainLoader= torch.utils.data.DataLoader(myNonFraudulentTrainDataset,batch_size=1,shuffle=True,num_workers=0)
nonFraudulentTestLoader= torch.utils.data.DataLoader(myNonFraudulentTestDataset,batch_size=1,shuffle=True,num_workers=0)

#make the trainloader and test loader for nonfraudulent dataset.
nonFraudulentValidationLoader= torch.utils.data.DataLoader(myNonFraudulentValidation,batch_size=1,shuffle=True,num_workers=0)
#make the trainloader and test loader for nonfraudulent dataset.
fraudulentTrainLoader= torch.utils.data.DataLoader(myFraudulentTrainDataset,batch_size=1,shuffle=True,num_workers=0)
fraudulentTestLoader= torch.utils.data.DataLoader(myFraudulentTestDataset,batch_size=1,shuffle=True,num_workers=0)



In [15]:
#Network architecture for the base autoencoders
class autoencoder1(nn.Module):
    def __init__(self):
        super(autoencoder1,self).__init__()
        self.encoder1= nn.Sequential(
            nn.Linear(97,72),
            nn.LeakyReLU(True),
            nn.Linear(72,48),
            nn.LeakyReLU(True),
            nn.Linear(48,25))
        
        self.decoder1= nn.Sequential(
            nn.Linear(25,48),
            nn.LeakyReLU(True),
            nn.Linear(48,72),
            nn.LeakyReLU(True),
            nn.Linear(72,97),nn.Softmax())
    
    def forward(self,x):
        x= self.encoder1(x)
        x= self.decoder1(x)
        return x;

In [16]:
model= autoencoder1()
criterion= nn.MSELoss()
optimizer= torch.optim.SGD(model.parameters(), lr=0.001,momentum=0.9)

In [17]:
#test the model
def testModel(loader):
    running_loss= 0.0
    for i,(feature,lable) in enumerate(loader):
        inputs= torch.tensor(feature)
        output= model(inputs)
        newoutput1= output.clone()
        
        #see the probability and make the value
        for l in range(newoutput1.shape[0]-4):
           
            if(newoutput1[l]>0.52):
                newoutput1[l]=1
            else:
                newoutput1[l]= 0
        
        
        output= newoutput1
        
        loss= criterion(output,inputs)
        running_loss += loss.item()
    
    return running_loss/2175
        

In [18]:
#train the model
for epoch in range(1000):
    validation_loss= []
    training_loss= []
    running_loss= 0.0
    for i, (feature,lable) in enumerate(nonFraudulentTrainLoader):
        
        #gets the inputs
        inputs= torch.tensor(feature)
        lables= torch.tensor(lable)
        lables= lables.type(torch.LongTensor)
        
        # =====================forward====================
        output = model(inputs)
       
        newoutput= output.clone()
        #print("input:",inputs)
        #print("output:",output)
               
       
        #see the probibility and make the output
        for k in range(newoutput.shape[1]-4):
            if(newoutput[0][k]>0.52):
                newoutput[0][k]=1
            else:
                newoutput[0][k]= 0
        
        #print("newoutput:",newoutput)
        #assert(False)
        output= newoutput
        
        #print("output:",output)
        #print(newoutput)
        loss= criterion(output,inputs)
     
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        # =======print the statistics
        running_loss += loss.item()
    
    print('[%d] loss: %.3f' %(epoch + 1,  running_loss /nonFraudX_trainNew.shape[0]))
    
    #test the model for the validation set.
    valid_loss= testModel(myNonFraudulentValidation)
    print("Validation Loss:",valid_loss)
    #append the loss after each epoch
    training_loss.append(running_loss/nonFraudX_trainNew.shape[0])
    validation_loss.append(valid_loss)

        

/usr/local/anaconda35/lib/python3.6/site-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


[1] loss: 0.194
Validation Loss: 0.19381802074525548
[2] loss: 0.194
Validation Loss: 0.1938114936461394
[3] loss: 0.194
Validation Loss: 0.19380467168216048
[4] loss: 0.194
Validation Loss: 0.19379751252031874
[5] loss: 0.194
Validation Loss: 0.19378996704501666
[6] loss: 0.194
Validation Loss: 0.193781978837375
[7] loss: 0.194
Validation Loss: 0.19377348426429705
[8] loss: 0.194
Validation Loss: 0.1937643962276393
[9] loss: 0.194
Validation Loss: 0.19375461311861017
[10] loss: 0.194
Validation Loss: 0.19374400647892348
[11] loss: 0.194
Validation Loss: 0.19373241685587783
[12] loss: 0.194
Validation Loss: 0.19371963431095254
[13] loss: 0.194
Validation Loss: 0.19370537749652206
[14] loss: 0.194
Validation Loss: 0.19368927664455327
[15] loss: 0.194
Validation Loss: 0.19367080948133578
[16] loss: 0.194
Validation Loss: 0.1936492172123372
[17] loss: 0.194
Validation Loss: 0.19362336767815996
[18] loss: 0.194
Validation Loss: 0.19359143901830433
[19] loss: 0.194
Validation Loss: 0.193550

[153] loss: 0.187
Validation Loss: 0.18685012787923044
[154] loss: 0.187
Validation Loss: 0.18684825092896648
[155] loss: 0.187
Validation Loss: 0.18684678902571228
[156] loss: 0.187
Validation Loss: 0.18684501367053766
[157] loss: 0.187
Validation Loss: 0.1868438623622916
[158] loss: 0.187
Validation Loss: 0.1868422483438733
[159] loss: 0.187
Validation Loss: 0.18684093365724058
[160] loss: 0.187
Validation Loss: 0.18683967098422435
[161] loss: 0.187
Validation Loss: 0.18683848796904773
[162] loss: 0.187
Validation Loss: 0.18683769939274625
[163] loss: 0.187
Validation Loss: 0.18683673483201826
[164] loss: 0.187
Validation Loss: 0.1868354165828091
[165] loss: 0.187
Validation Loss: 0.1868344230624451
[166] loss: 0.187
Validation Loss: 0.18683377822925304
[167] loss: 0.187
Validation Loss: 0.18683270920282122
[168] loss: 0.187
Validation Loss: 0.1868320564223432
[169] loss: 0.187
Validation Loss: 0.1868311611879831
[170] loss: 0.187
Validation Loss: 0.18683041136840295
[171] loss: 0.18

[303] loss: 0.187
Validation Loss: 0.18680868453678043
[304] loss: 0.187
Validation Loss: 0.1868086377094532
[305] loss: 0.187
Validation Loss: 0.18680863968942357
[306] loss: 0.187
Validation Loss: 0.1868084644106613
[307] loss: 0.187
Validation Loss: 0.1868085401702201
[308] loss: 0.187
Validation Loss: 0.18680835747855834
[309] loss: 0.187
Validation Loss: 0.18680824653855685
[310] loss: 0.187
Validation Loss: 0.18680830025124823
[311] loss: 0.187
Validation Loss: 0.18680825578070234
[312] loss: 0.187
Validation Loss: 0.18680808717491984
[313] loss: 0.187
Validation Loss: 0.18680797587866071
[314] loss: 0.187
Validation Loss: 0.186807919117226
[315] loss: 0.187
Validation Loss: 0.18680792109034527
[316] loss: 0.187
Validation Loss: 0.1868078013055626
[317] loss: 0.187
Validation Loss: 0.1868078246678429
[318] loss: 0.187
Validation Loss: 0.18680806455255924
[319] loss: 0.187
Validation Loss: 0.18680769533261485
[320] loss: 0.187
Validation Loss: 0.18680794864550404
[321] loss: 0.187

[453] loss: 0.187
Validation Loss: 0.1867925182019157
[454] loss: 0.187
Validation Loss: 0.18679231580646558
[455] loss: 0.187
Validation Loss: 0.18679198199990152
[456] loss: 0.187
Validation Loss: 0.18679170607150286
[457] loss: 0.187
Validation Loss: 0.18679149296777
[458] loss: 0.187
Validation Loss: 0.1867912535009713
[459] loss: 0.187
Validation Loss: 0.18679113875860456
[460] loss: 0.187
Validation Loss: 0.1867907889478508
[461] loss: 0.187
Validation Loss: 0.18679053134616763
[462] loss: 0.187
Validation Loss: 0.18679037999832768
[463] loss: 0.187
Validation Loss: 0.18679011403829202
[464] loss: 0.187
Validation Loss: 0.1867901279665958
[465] loss: 0.187
Validation Loss: 0.1867895975880239
[466] loss: 0.187
Validation Loss: 0.1867894375118716
[467] loss: 0.187
Validation Loss: 0.18678958183047414
[468] loss: 0.187
Validation Loss: 0.18678894978145072
[469] loss: 0.187
Validation Loss: 0.18678857534096158
[470] loss: 0.187
Validation Loss: 0.18678837933074469
[471] loss: 0.187
V

[603] loss: 0.187
Validation Loss: 0.18674599594083324
[604] loss: 0.187
Validation Loss: 0.1867458180723519
[605] loss: 0.187
Validation Loss: 0.18674570568676652
[606] loss: 0.187
Validation Loss: 0.1867454373630984
[607] loss: 0.187
Validation Loss: 0.1867453857057396
[608] loss: 0.187
Validation Loss: 0.18674515313800724
[609] loss: 0.187
Validation Loss: 0.18674503579221924
[610] loss: 0.187
Validation Loss: 0.18674476660531142
[611] loss: 0.187
Validation Loss: 0.1867445343664323
[612] loss: 0.187
Validation Loss: 0.18674454151898964
[613] loss: 0.187
Validation Loss: 0.1867443030456017
[614] loss: 0.187
Validation Loss: 0.18674413485088567
[615] loss: 0.187
Validation Loss: 0.18674389291083676
[616] loss: 0.187
Validation Loss: 0.1867437594238369
[617] loss: 0.187
Validation Loss: 0.18674358969447258
[618] loss: 0.187
Validation Loss: 0.18674334184876804
[619] loss: 0.187
Validation Loss: 0.18674318634915626
[620] loss: 0.187
Validation Loss: 0.1867430140574773
[621] loss: 0.187

[753] loss: 0.187
Validation Loss: 0.1867343804822571
[754] loss: 0.187
Validation Loss: 0.1867343502825704
[755] loss: 0.187
Validation Loss: 0.1867342941514377
[756] loss: 0.187
Validation Loss: 0.18673426280076477
[757] loss: 0.187
Validation Loss: 0.18673425788166878
[758] loss: 0.187
Validation Loss: 0.18673438390096028
[759] loss: 0.187
Validation Loss: 0.1867341918780886
[760] loss: 0.187
Validation Loss: 0.18673417844306464
[761] loss: 0.187
Validation Loss: 0.18673420062695426
[762] loss: 0.187
Validation Loss: 0.18673413914510573
[763] loss: 0.187
Validation Loss: 0.1867340998951046
[764] loss: 0.187
Validation Loss: 0.1867342801272184
[765] loss: 0.187
Validation Loss: 0.18673406716050772
[766] loss: 0.187
Validation Loss: 0.18673444414275817
[767] loss: 0.187
Validation Loss: 0.1867340405782064
[768] loss: 0.187
Validation Loss: 0.1867340995868047
[769] loss: 0.187
Validation Loss: 0.18673399018145156
[770] loss: 0.187
Validation Loss: 0.18673403937241126
[771] loss: 0.187


[903] loss: 0.187
Validation Loss: 0.18673140783419553
[904] loss: 0.187
Validation Loss: 0.18673144154164983
[905] loss: 0.187
Validation Loss: 0.18673155556465018
[906] loss: 0.187
Validation Loss: 0.186731659776863
[907] loss: 0.187
Validation Loss: 0.18673140130508906
[908] loss: 0.187
Validation Loss: 0.18673139546794454
[909] loss: 0.187
Validation Loss: 0.18673142133087947
[910] loss: 0.187
Validation Loss: 0.18673132002353668
[911] loss: 0.187
Validation Loss: 0.186731377264549
[912] loss: 0.187
Validation Loss: 0.18673134518765855
[913] loss: 0.187
Validation Loss: 0.18673155612644107
[914] loss: 0.187
Validation Loss: 0.1867313177215642
[915] loss: 0.187
Validation Loss: 0.18673153014703728
[916] loss: 0.187
Validation Loss: 0.18673136090410167
[917] loss: 0.187
Validation Loss: 0.18673126411849056
[918] loss: 0.187
Validation Loss: 0.1867312127351761
[919] loss: 0.187
Validation Loss: 0.18673120790514453
[920] loss: 0.187
Validation Loss: 0.18673118500873961
[921] loss: 0.18

In [23]:
#convert list to dataframe
print("validationLoss:",len(validation_loss))
print("trainingLoss:",len(training_loss))

validLossFrame= pd.DataFrame({'loss':validation_loss})
trainingLossFrame= pd.DataFrame({'loss':training_loss})




validationLoss: 1
trainingLoss: 1


In [24]:
validLossFrame.head()

,loss
0,0.18673


In [84]:
#function for testing the model
def testModel2(loader):
    running_loss= 0.0
    predictedLabel= []
    lossList= []
    for i,(feature,lable) in enumerate(loader):
        inputs= torch.tensor(feature)
        output= model(inputs)
        newoutput1= output.clone()
        
        #see the probability and make the value
        for l in range(newoutput1.shape[0]-4):
           
            if(newoutput1[l]>0.52):
                newoutput1[l]=1
            else:
                newoutput1[l]= 0
        
        
        output= newoutput1
        
        loss= criterion(output,inputs)
        lossList.append(loss.item())
        if(loss.item()<0.189):
            predictedLabel.append(0)
        else:
            predictedLabel.append(1)
            
    
    return predictedLabel,lossList
        

In [85]:
#test the model on test set
predictedLable1,lossList1= testModel2(myNonFraudulentTestDataset)

/usr/local/anaconda35/lib/python3.6/site-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [86]:
#count the predicted lables
class1= predictedLable1.count(1)
class0= predictedLable1.count(0)

In [87]:
print(class1)
print(class0)

355
3269


In [88]:
#test the model on test set
predictedLable2,lossList2= testModel2(myFraudulentTestDataset)


/usr/local/anaconda35/lib/python3.6/site-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [89]:
#count the predicted lables
class1= predictedLable2.count(1)
class0= predictedLable2.count(0)

In [90]:
print(class1)
print(class0)

20
211


In [91]:
#convert loss to dataframe

trainLoss= pd.DataFrame({'loss':lossList1})
testLoss= pd.DataFrame({'loss':lossList2})

In [92]:
trainLoss.to_csv('trainLoss.csv',header=None,sep='\t')
testLoss.to_csv('testLoss.csv',header=None,sep='\t')